In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

# Define the VGG class
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.chosen_features = ['0', '5', '10', '19', '28']
        self.model = models.vgg19(pretrained=True).features[:29]  # from 0 to 28

    def forward(self, x):
        features = []
        for layer_num, layer in enumerate(self.model):
            x = layer(x)
            if str(layer_num) in self.chosen_features:
                features.append(x)
        return features

    def load_image(self, image_name):
        image = Image.open(image_name)
        image = loader(image).unsqueeze(0)
        return image.to(device)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 356

# Define image transformations
loader = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ]
)

# Create an instance of the VGG class
model = VGG().to(device).eval()

# Load images
original_img = model.load_image("images/my_image.jpg")
style_img = model.load_image("images/style.jpg")

# Initialize generated image
generated = original_img.clone().requires_grad_(True)

# Hyperparameters
total_steps = 2000
lr = 0.001
alpha = 1  # content loss
beta = 0.01  # how much style we want in the image
optimizer = optim.Adam([generated], lr=lr)

# Training loop
for step in range(total_steps):
    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)

    style_loss = original_loss = 0

    for gen_feature, orig_feature, style_feature in zip(
        generated_features, original_img_features, style_features
    ):
        batch_size, channel, height, width = gen_feature.shape
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)

        # Compute Gram Matrix
        G = gen_feature.view(channel, height * width).mm(
            gen_feature.view(channel, height * width).t()
        )

        A = style_feature.view(channel, height * width).mm(
            style_feature.view(channel, height * width).t()
        )
        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(total_loss)
        save_image(generated, "images/generated.png")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:03<00:00, 173MB/s]


tensor(594714.5625, device='cuda:0', grad_fn=<AddBackward0>)
tensor(32857.2422, device='cuda:0', grad_fn=<AddBackward0>)
tensor(17207.5742, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9847.1133, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6184.3174, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4507.3945, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3587.2129, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2967.3860, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2507.3296, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2150.1777, device='cuda:0', grad_fn=<AddBackward0>)


In [2]:
import shutil

# Replace 'my_folder' with the name of your folder
shutil.make_archive('images', 'zip', 'images')

'/content/images.zip'

In [4]:
from google.colab import files

files.download('images.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>